In [ ]:
import pandas as pd
import collections
import math
import os

import cv2
import matplotlib.pyplot as plt
import cv2
import sys
import numpy as np
import math
import time
from PIL import Image 

from pandas import read_csv  
import configparser

config = configparser.ConfigParser()
config.read('config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

window_size = int(config.get('Configuration', 'window_size'))

scenes=config.get('Configuration', 'scenes')
scenes = [(item.strip()) for item in scenes.split(',')]

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

other_users=config.get('Configuration', 'other_users')
other_users = [(item.strip()) for item in other_users.split(',')]

anchors=config.get('Configuration', 'anchors')
anchors = [(item.strip()) for item in anchors.split(',')]

other_anchors=config.get('Configuration', 'other_anchors')
other_anchors = [(item.strip()) for item in other_anchors.split(',')]

anchors.extend(other_anchors)

room=config.get('Configuration', 'room')

w = int(config.get(room, 'w'))
h = int(config.get(room, 'h'))

user_color={}
for user in users:
    user_color[user]=config.get('Colors', user)
print(user_color)


In [ ]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

days=list(range(day_time(t0),day_time(tN)+1))
print(days)

i0=0
ts=list(range(t0,tN,1))
print(i0,ts)

In [ ]:
scene_user = {}
for d in days:
    for user in users:
        scene_user[user] = {}
        for s in scenes:
            print(d,source_folder+"/"+s+"/location_gt_"+user+"/"+str(d)+".location.tsv")
            try:
                series=read_csv(source_folder+"/"+s+"/location_gt_"+user+"/"+str(d)+".location.tsv", sep="\t",parse_dates=True, header=None, names=["time","date","time2","x","y"])
                print(d,series.size,(series["time"].iloc[0],series["time"].iloc[-1]))
                scene_user[user][s] = (series["time"].iloc[0],series["time"].iloc[-1])
                print(scene_user[user][s])
            except Exception as e:
                print("Not data",e)
                continue
def getScene(t0):
    for user, user_scene in scene_user.items():
        for scene, times in user_scene.items():
            #print(t0,times[0],times[1])
            if(t0>times[0]-window_size/2 and t0<times[1]+window_size/2):
                return scene
    return None

In [ ]:
def getFPImg(tt):
    return np.array(Image.open(data_folder+"/"+room+"/DAY_"+str(day)+"/FP."+str(tt)+".png"))/255.0

def getGTImg(tt,user):
    return np.array(Image.open(data_folder+"/"+room+"/DAY_"+str(day)+"/GT."+user+"."+str(tt)+".png"))/255.0    

In [ ]:

XY0={}
rssi0={}
Y1={}
Y0={}

T0=[]
T1=[]

for day in days:
    anchors_data={}
    for user in users:
        XY0[user]=[]
        rssi0[user]=[]
        Y1[user]=[]
        Y0[user]=[]


        anchors_data[user]={}
        for anchor in anchors:
            print(data_folder+"/"+room+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv")
            print(user,anchor)
            series=read_csv(data_folder+"/"+room+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv", sep="\t",parse_dates=True, header=None, names=["time","d"])
            series=series.values
            anchors_data[user][anchor]=series
            

    for it,tt in enumerate(ts):
        imgX=getFPImg(tt)
        print("imgX",imgX.shape)

        for user in users:
            
            imgY=getGTImg(tt,user)
            print("user",user,"imgY",imgY.shape)

            dist=[]
            for ixa,anchor in enumerate(anchors):
                d=anchors_data[user][anchor][it][1]
                if(d>0):
                    d=d/10.0
                if(d>1):
                    d=1
                print("\t anchor:",anchor, anchors_data[user][anchor][it], "tt:",tt)
                dist.append(d)

            dist=np.array(dist)
            print("dist:",dist)

            rssi0[user].append(dist.reshape(len(anchors),1))
            XY0[user].append(imgX[:, :, np.newaxis])
            T0.append(it)
            T1.append(tt)
            Y1[user].append("1")
            Y0[user].append(imgY[:, :, np.newaxis])
        
T0=list(dict.fromkeys(T0))
T1=list(dict.fromkeys(T1)) 


In [ ]:
for day in days:
    anchors_data={}
    for user in other_users:
        XY0[user]=[]
        rssi0[user]=[]
        Y1[user]=[]
        Y0[user]=[]

        anchors_data[user]={}
        for anchor in anchors:
            print(data_folder+"/"+room+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv")
            print(user,anchor)
            series=read_csv(data_folder+"/"+room+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv", sep="\t",parse_dates=True, header=None, names=["time","d"])
            series=series.values
            anchors_data[user][anchor]=series
            
    for it,tt in enumerate(ts):
        imgX=getFPImg(tt)
        print("imgX",imgX.shape)

        for user in other_users:
            
            #imgY=getGTImg(tt,user)
            #print("user",user,"imgY",imgY.shape)
            
            dist=[]
            for ixa,anchor in enumerate(anchors):
                d=anchors_data[user][anchor][it][1]
                if(d>0):
                    d=d/10.0
                if(d>1):
                    d=1
                print("\t anchor:",anchor, anchors_data[user][anchor][it], "tt:",tt)
                dist.append(d)

            dist=np.array(dist)
            print("dist:",dist)

            rssi0[user].append(dist.reshape(len(anchors),1))
            XY0[user].append(imgX[:, :, np.newaxis])
            T0.append(it)
            T1.append(tt)
            Y1[user].append("0")      
            #Y0[user].append(imgY[:, :, np.newaxis])

In [ ]:
for user in users:
    XY0[user]=np.array(XY0[user])
    rssi0[user]=np.array(rssi0[user])
    Y0[user]=np.array(Y0[user])
for user in other_users:
    XY0[user]=np.array(XY0[user])
    rssi0[user]=np.array(rssi0[user])

T0=np.array(T0)
T1=np.array(T1)


In [ ]:

print(rssi0)

In [ ]:
minT=np.min(T0)
maxT=np.max(T0)

#print(XY0.shape)
#print(rssi0.shape)

print(minT)
print(maxT)

In [ ]:
XYT=[]
rssiT=[]
YT=[]

TA=int(window_size/2)
TB=int(window_size/2)

#T1Y=3
#T2Y=3

T=window_size

S=[]

for user in users:
    for t in range(minT+TA,maxT-TB):

        
        tt=T1[t]
        scene=getScene(tt)
        print("Scene",scene,tt,t)
        if not scene:
            continue
                    
        label=Y1[user][t]

        subW=list(range(t-TA,t+TB))
        print("subW:",subW, "t:",t)
        if not (set(subW).intersection(set(T0)) == set(subW)):
            print("Not data for sliding window!",t)
            continue

        print("rssi0[t-T:t+T]",rssi0[user])
        if(all(map(lambda x: x == -1, rssi0[user][t-TA:t+TB].flatten()))):
            print("Distance -1 for RSSI in sliding window!",t)
            continue

        #if(all(map(lambda x: x == -1, rssi0[t:t+T2].flatten()))):
        #    print("Distance -1 for RSSI in sliding window!",t)
        #    continue

        if(all(map(lambda x: x == 0, Y0[user][t-TA:t+TB].flatten()))):
            print("Distance 0 for Y0 in t!",time2str(tt))
            #label="0"     

        if(XY0[user][t-TA:t+TB].shape[0]!=T):
            print("#",XY0[user][t-TA:t+TB].shape)
            continue
        if(rssi0[user][t-TA:t+TB].shape[0]!=T):
            print("@",rssi0[user][t-TA:t+TB].shape)
            continue

        XYT.append(XY0[user][t-TA:t+TB])
        rssiT.append(rssi0[user][t-TA:t+TB])
        YT.append(label)
        S.append(scene)
        
for user in other_users:
    for t in range(minT+TA,maxT-TB):
        tt=T1[t]
        scene=getScene(tt)
        print("Scene",scene,tt,t)
        if not scene:
            continue
            
        label=Y1[user][t]

        subW=list(range(t-TA,t+TB))
        print("subW:",subW, "t:",t)
        if not (set(subW).intersection(set(T0)) == set(subW)):
            print("Not data for sliding window!",t)
            continue

        print("rssi0[t-T:t+T]",rssi0[user][t-TA:t+TB].flatten())
        if(all(map(lambda x: x == -1, rssi0[user][t-TA:t+TB].flatten()))):
            print("Distance -1 for RSSI in sliding window!",t)
            continue

        #if(all(map(lambda x: x == -1, rssi0[t:t+T2].flatten()))):
        #    print("Distance -1 for RSSI in sliding window!",t)
        #    continue

  


        if(XY0[user][t-TA:t+TB].shape[0]!=T):
            print("#",XY0[user][t-TA:t+TB].shape)
            continue
        if(rssi0[user][t-TA:t+TB].shape[0]!=T):
            print("@",rssi0[user][t-TA:t+TB].shape)
            continue

        XYT.append(XY0[user][t-TA:t+TB])
        rssiT.append(rssi0[user][t-TA:t+TB])
        YT.append(label)
        S.append(scene)        

XYT=np.array(XYT)
rssiT=np.array(rssiT)
YT=np.array(YT)    
S=np.array(S)    

print(XYT.shape)
print(rssiT.shape)
print(YT.shape)
print(S.shape)


In [ ]:
print(YT)

In [ ]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Softmax,Input, Conv1D, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D
from tensorflow.keras.models import Model

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from keras.losses import mean_squared_error

from keras import backend as K

from sklearn.model_selection import KFold

from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Lambda, Dot,Reshape,Multiply, Input, Conv1D, Conv2D, ConvLSTM2D,ReLU,UpSampling2D, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D, Concatenate
from tensorflow.keras.models import Model, Sequential

from keras import backend as K
import math

from tensorflow.keras.losses import CategoricalCrossentropy

In [ ]:
K=len(anchors)
print(K)
print(T)
rssi = Input(shape=(T,K,1))
print(rssi)

encoded_rssi = TimeDistributed(Conv1D(32, 3, activation='relu', padding='same',
        input_shape=(T, K, 1)))(rssi)
#encoded_rssi = TimeDistributed(MaxPooling1D(2, padding='same'))(encoded_rssi)
encoded_rssi = TimeDistributed(Conv1D(64, 3, activation='relu', padding='same'))(encoded_rssi)
print(encoded_rssi)
encoded_rssi = TimeDistributed(Dropout(0.25))(encoded_rssi)
encoded_rssi = TimeDistributed(Flatten())(encoded_rssi)

encoded_rssi=LSTM(128, return_sequences=True)(encoded_rssi)
encoded_rssi = Dropout(0.25)(encoded_rssi)
encoded_rssi=LSTM(128, return_sequences=True)(encoded_rssi)
print("lstm",encoded_rssi)
encoded_rssi=Reshape((T,1,1,128))(encoded_rssi)

print("lstm",encoded_rssi)

In [ ]:
N=w
M=h

xy = Input(shape=(T,N,M,1))
print(xy)

encoded = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'),
        input_shape=(T, N, M, 1))(xy)
encoded = TimeDistributed(MaxPooling2D((2, 2), padding='same'))(encoded)
encoded = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same'))(encoded)
encoded = TimeDistributed(Dropout(0.25))(encoded)
#encoded = TimeDistributed(MaxPooling2D((2, 2), padding='same'))(encoded)
#encoded = TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same'))(encoded)
#encoded = TimeDistributed(MaxPooling2D((2, 2), padding='same'))(encoded)
#encoded = TimeDistributed(Conv2D(256, (3, 3), activation='relu', padding='same'))(encoded)
print(encoded)

#LayerNormalization
encoded= ConvLSTM2D(128 , 3,return_sequences=True, padding='same')(encoded)
encoded = Dropout(0.25)(encoded)
encoded= ConvLSTM2D(128 , 3,return_sequences=True, padding='same')(encoded)


print(encoded)


In [ ]:
import tensorflow as tf
#merged=Multiply()([encoded, encoded_rssi])
encoded_rssi2=Reshape((4,128))(encoded_rssi)
print(encoded_rssi2)
print(encoded)
merged=Dot(axes=[4, 2])([encoded, encoded_rssi2])
merged=Softmax()(merged)
# Create a lambda layer to expand dimensions of input2
#expand_dim = Lambda(lambda x: tf.repeat(x, 8, axis=3))(encoded_rssi)  # Repeat along axis 2
#merged=Concatenate(axis=2)([encoded, encoded_rssi])

print(merged)

In [ ]:
dense=  Flatten()(merged)
dense = Dropout(0.25)(dense)
dense = Dense(1024)(dense)
dense = Dropout(0.25)(dense)
dense = Dense(512)(dense)
dense = Dense(2, activation="softmax")(dense)

# Compile model
autoencoder = Model([xy,rssi], dense)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#autoencoder.compile(optimizer=Adam(learning_rate=0.001),
#              loss=CategoricalCrossentropy(),
#              metrics=['accuracy'])

In [ ]:
from keras.models import load_model

print(encoded)
autoencoder.summary()
autoencoder.save("autoencoder.room.h5")

In [ ]:
from keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder

# Use LabelEncoder to convert string labels to numerical labels
label_encoder = LabelEncoder()
numerical_labels = label_encoder.fit_transform(YT)

YT_categorical=to_categorical(YT)

In [ ]:
np.random.seed(42)



In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter

Y_pred_total=np.array([])

print(np.unique(S))

total_execution_time = 0

print(label_encoder.classes_)
def create_class_weight(labels_dict,label_encoder,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    class_weight = dict()
    
    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[label_encoder.transform([key])[0]] = score if score > 1.0 else 1.0
    
    return class_weight

for ixs,sc in enumerate(np.unique(S)):
    
    #obtenemos train and test index basado en la escena ( y luego lo barajamos)
    train_indices=np.where(S != sc)[0]
    test_indices=np.where(S == sc)[0]
    
    np.random.shuffle(train_indices)

    #datos de train y test
    print("SCENE", sc, " TRAIN:", train_indices, " TEST:", test_indices)
    
    XYT_train=XYT[train_indices].copy()
    rssiT_train=rssiT[train_indices].copy()
    YT_train=YT_categorical[train_indices].copy()
    YT_train0=YT[train_indices].copy()

    XYT_test=XYT[test_indices].copy()
    rssiT_test=rssiT[test_indices].copy()
    YT_test=YT_categorical[test_indices].copy()
    
    autoencoder = load_model("autoencoder.room.h5")
    class_labels = np.unique(YT_train0)
    print(class_labels)

    class_weights = compute_class_weight(class_weight="balanced",
                                         classes=np.unique(YT_train0),
                                         y=YT_train0)
    
    print(class_weights)
    # Create a dictionary with class labels as keys and corresponding weights as values
    class_weight_dict = {label_encoder.transform([class_label])[0]: weight for class_label, weight in zip(class_labels, class_weights)}

    # Print the dictionary
    print(class_weight_dict)

    print("Counter(YT_train)",Counter(YT_train0))
    #class_weight_dict =create_class_weight(Counter(YT_train0),YT_train0)
    #print(class_weight_dict)
    
    start_time = time.time()
    
    history = autoencoder.fit([XYT_train,rssiT_train] , YT_train, batch_size=16, class_weight=class_weight_dict,validation_data=([XYT_test,rssiT_test] , YT_test),epochs=25)
    
    end_time = time.time()
    training_time = end_time - start_time
    
    y_pred = autoencoder.predict([XYT_test,rssiT_test]).argmax(axis=1)
    y_val=YT_test.argmax(axis=1)
    print(y_pred)
    print(y_val)

    cm = confusion_matrix(y_val, y_pred) 
    print(cm)
    print("training time:",training_time)
    print(classification_report(y_val, y_pred, target_names=["out","in"]))



    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["out","in"])

    disp.plot(cmap=plt.cm.Blues)
    plt.show()
    
    Y_pred_total=np.concatenate((Y_pred_total,y_pred))
    
    total_execution_time = total_execution_time+training_time
    
    

print("Total training time:",total_execution_time)
